In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

* y = Prediction (1, -1)
* tX = values of 30 features
* ids = unique id of each event

## Data Backgound

The Higgs boson is an elementary particle in the Standard Model of physics which explains why other particles have mass. Its discovery at the Large Hadron Collider at CERN was announced in March 2013. In this project, you will apply machine learning techniques to actual CERN particle accelerator data to recreate the process of “discovering” the Higgs particle. For some background, physicists at CERN smash protons into one another at high speeds to generate even smaller particles as by-products of the collisions. Rarely, these collisions can produce a Higgs boson. Since the Higgs boson decays rapidly into other particles, scientists don’t observe it directly, but rather measure its“decay signature”, or the products that result from its decay process. Since many decay signatures look similar, it is our job to estimate the likelihood that a given event’s signature was the result of a Higgs boson (signal) or some other process/particle (background). In practice, this means that you will be given a vector of features representing the decay signature of a collision event, and asked to predict whether this event was signal (a Higgs boson) or background (something else). To do this, you will use the binary classification techniques we have discussed in the lectures.

Further Information: https://higgsml.lal.in2p3.fr/files/2014/04/documentation_v1.8.pdf.

## Exploring the data

There are:
- 250, 000 data points
- 30 different sets of features

In [3]:
print(y.shape, tX.shape, ids.shape)

(250000,) (250000, 30) (250000,)


In [4]:
HEADERS = ['Id', 'Prediction', 'DER_mass_MMC', 'DER_mass_transverse_met_lep',
       'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet',
       'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_deltar_tau_lep',
       'DER_pt_tot', 'DER_sum_pt', 'DER_pt_ratio_lep_tau',
       'DER_met_phi_centrality', 'DER_lep_eta_centrality', 'PRI_tau_pt',
       'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta',
       'PRI_lep_phi', 'PRI_met', 'PRI_met_phi', 'PRI_met_sumet',
       'PRI_jet_num', 'PRI_jet_leading_pt', 'PRI_jet_leading_eta',
       'PRI_jet_leading_phi', 'PRI_jet_subleading_pt',
       'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi',
       'PRI_jet_all_pt']

In [5]:
N, M = tX.shape

#### Lets see how the data (x, y) looks like

In [6]:
def print_topN(table, N):
    top_ten = []
    for i in range(0, N):
        top_ten.append(table[i])
    print(np.array(top_ten))

In [7]:
print_topN(tX, 5)

[[ 1.38470e+02  5.16550e+01  9.78270e+01  2.79800e+01  9.10000e-01
   1.24711e+02  2.66600e+00  3.06400e+00  4.19280e+01  1.97760e+02
   1.58200e+00  1.39600e+00  2.00000e-01  3.26380e+01  1.01700e+00
   3.81000e-01  5.16260e+01  2.27300e+00 -2.41400e+00  1.68240e+01
  -2.77000e-01  2.58733e+02  2.00000e+00  6.74350e+01  2.15000e+00
   4.44000e-01  4.60620e+01  1.24000e+00 -2.47500e+00  1.13497e+02]
 [ 1.60937e+02  6.87680e+01  1.03235e+02  4.81460e+01 -9.99000e+02
  -9.99000e+02 -9.99000e+02  3.47300e+00  2.07800e+00  1.25157e+02
   8.79000e-01  1.41400e+00 -9.99000e+02  4.20140e+01  2.03900e+00
  -3.01100e+00  3.69180e+01  5.01000e-01  1.03000e-01  4.47040e+01
  -1.91600e+00  1.64546e+02  1.00000e+00  4.62260e+01  7.25000e-01
   1.15800e+00 -9.99000e+02 -9.99000e+02 -9.99000e+02  4.62260e+01]
 [-9.99000e+02  1.62172e+02  1.25953e+02  3.56350e+01 -9.99000e+02
  -9.99000e+02 -9.99000e+02  3.14800e+00  9.33600e+00  1.97814e+02
   3.77600e+00  1.41400e+00 -9.99000e+02  3.21540e+01 -7.050

In [8]:
import collections

freq = collections.Counter()
for x in y:
    freq[x] += 1
print(freq)

Counter({-1.0: 164333, 1.0: 85667})


#### 181,886 rows out the 250,000 have undefined values

In [9]:
undefined_count = 0
for row in tX:
    for v in row:
        if v == -999.0:
            undefined_count+=1
            break
print(undefined_count)

181886


In [10]:
undefined_st = {}
for i in range(0, M): undefined_st[i] = 0
for row in tX:
    for i in range(0, M):
        if row[i] == -999.0:
            undefined_st[i] += 1

# TODO: Why not maka nice graph for below :)
print("Undefined values for each feature: \n")
for i in range(0, M):
    print("{0} : {1} / {2}".format(HEADERS[i+2], undefined_st[i], N - undefined_st[i]))

Undefined values for each feature: 

DER_mass_MMC : 38114 / 211886
DER_mass_transverse_met_lep : 0 / 250000
DER_mass_vis : 0 / 250000
DER_pt_h : 0 / 250000
DER_deltaeta_jet_jet : 177457 / 72543
DER_mass_jet_jet : 177457 / 72543
DER_prodeta_jet_jet : 177457 / 72543
DER_deltar_tau_lep : 0 / 250000
DER_pt_tot : 0 / 250000
DER_sum_pt : 0 / 250000
DER_pt_ratio_lep_tau : 0 / 250000
DER_met_phi_centrality : 0 / 250000
DER_lep_eta_centrality : 177457 / 72543
PRI_tau_pt : 0 / 250000
PRI_tau_eta : 0 / 250000
PRI_tau_phi : 0 / 250000
PRI_lep_pt : 0 / 250000
PRI_lep_eta : 0 / 250000
PRI_lep_phi : 0 / 250000
PRI_met : 0 / 250000
PRI_met_phi : 0 / 250000
PRI_met_sumet : 0 / 250000
PRI_jet_num : 0 / 250000
PRI_jet_leading_pt : 99913 / 150087
PRI_jet_leading_eta : 99913 / 150087
PRI_jet_leading_phi : 99913 / 150087
PRI_jet_subleading_pt : 177457 / 72543
PRI_jet_subleading_eta : 177457 / 72543
PRI_jet_subleading_phi : 177457 / 72543
PRI_jet_all_pt : 0 / 250000


### Load necessary helper functions

In [11]:
from helper_funcs import *      # MSE, gradient descent, build_poly etc...
from implementations import *   # implemented regression algorithm

#### TODO: Think about how to deal with these (-999) values. Some ideas:

- Drop the row 
- Drop the feature away
- Fill them with the average
- Predict a reasonable value for them?

### Data pre-processing

In [12]:
def cleanData(tx):
    '''
    Set outliers with threshold 990 to 0 in the data set.
    '''
    x_clean = np.c_[np.ones(tx.shape[0]), tx]
    
    for i in range(x_clean.shape[0]):
        for j in range(x_clean.shape[1]):
            # filter out outlier with 0
            if x_clean[i, j] <= -990:
                x_clean[i, j] = 0
    return x_clean

### Function to compute accuracy

In [13]:
def compute_accuracy(pred, actual):
    return np.sum(pred == actual) / len(actual)

### TODO: Engineer new features from the existing set of features

### TODO: Find out how to measure the importance of each feature? e.g. a heatmap of how correlated the features are to the y value

## Do your thing crazy machine learning thing here :) ...

## Making predictions with the model

### 1. Least square method with cross validation

In [14]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

In [21]:
def cross_validation(y, tx, k, k_indices):
    # Get the kth test set
    k_test = k_indices[k]
    
    # Collect remaining data as training set & flatten the array
    mask = np.ones(len(k_indices), dtype=bool)
    mask[[k]] = False
    k_train = k_indices[mask].ravel()
    
    # Split the data set
    train_set_x = tx[k_train, :]
    test_set_x = tx[k_test, :]
    
    train_set_y = y[k_train]
    test_set_y = y[k_test]
    
    train_set_x = cleanData(train_set_x)
    test_set_x = cleanData(test_set_x)
    
    # Calculate weight and loss using least square method
    w, loss = least_squares(train_set_y, train_set_x)
    
    # Predict for the output
    train_pred_y = predict_labels(w, train_set_x)
    test_pred_y = predict_labels(w, test_set_x)
    
    # Calculate prediction accuracy
    train_acc = compute_accuracy(train_pred_y, train_set_y)
    test_acc = compute_accuracy(test_pred_y, test_set_y)
    
    return train_acc, test_acc

In [22]:
k_fold = 4
seed = 1

# split training data into k folds
k_indices = build_k_indices(y, k_fold, seed)

results_train = []
results_test = []

for k in range(k_fold):
    result_train, result_test = cross_validation(y, tX, k, k_indices)
    
    results_train.append(result_train)
    results_test.append(result_test)
    
for i in range(k_fold):
    print("{ind} - Train Accuracy: {a:.6f} || Test Accuracy: {b:.6f}".format(
           ind=i, a=results_train[i], b=results_test[i]))
    
print("Average test accuracy: {ta:.6f}".format(ta=np.mean(results_test)))
print("Variance of test accuracy: {tv:.6f}".format(tv=np.var(results_test)))

0 - Train Accuracy: 0.746667 || Test Accuracy: 0.747280
1 - Train Accuracy: 0.746725 || Test Accuracy: 0.745008
2 - Train Accuracy: 0.745899 || Test Accuracy: 0.748384
3 - Train Accuracy: 0.746581 || Test Accuracy: 0.745376
Average test accuracy: 0.746512
Variance of test accuracy: 0.000002


### 2. Ridge regression with cross validation

In [23]:
def cv_ridge_regression(y, tx, k, k_indices, degree, lambda_):
    # Get the kth test set
    k_test = k_indices[k]
    
    # Collect remaining data as training set & flatten the array
    mask = np.ones(len(k_indices), dtype=bool)
    mask[[k]] = False
    k_train = k_indices[mask].ravel()
    
    # Split the data set
    train_set_x = tx[k_train, :]
    test_set_x = tx[k_test, :]
    
    train_set_y = y[k_train]
    test_set_y = y[k_test]
    
    # Build polynomial basis function
    train_set_x = build_poly(train_set_x, degree)
    test_set_x = build_poly(test_set_x, degree)
    
    # Calculate weight using ridge regression method
    w, loss = ridge_regression(train_set_y, train_set_x, lambda_)
    
    # Predict for the output
    train_pred_y = predict_labels(w, train_set_x)
    test_pred_y = predict_labels(w, test_set_x)
    
    # Calculate prediction accuracy
    train_acc = compute_accuracy(train_pred_y, train_set_y)
    test_acc = compute_accuracy(test_pred_y, test_set_y)
    
    return train_acc, test_acc

In [24]:
k_fold = 4
seed = 10

degree = 7
lambda_ = 0.001

k_indices = build_k_indices(y, k_fold, seed)

results_train = []
results_test = []

for k in range(k_fold):
    result_train, result_test = cv_ridge_regression(y, tX, k, k_indices, degree, lambda_)
    
    results_train.append(result_train)
    results_test.append(result_test)
    
for i in range(k_fold):
    print("{ind} - Train Accuracy: {a:.6f} || Test Accuracy: {b:.6f}".format(
           ind=i, a=results_train[i], b=results_test[i]))
    
print("Average test accuracy: {ta:.6f}".format(ta=np.mean(results_test)))
print("Variance of test accuracy: {tv:.6f}".format(tv=np.var(results_test)))

0 - Train Accuracy: 0.791413 || Test Accuracy: 0.789152
1 - Train Accuracy: 0.566416 || Test Accuracy: 0.569456
2 - Train Accuracy: 0.800475 || Test Accuracy: 0.798896
3 - Train Accuracy: 0.582928 || Test Accuracy: 0.581184
Average test accuracy: 0.684672
Variance of test accuracy: 0.011987


[Further optimisation] We can also use logistic regression to tackle this problem. Following which, we can try and move on and try other models.

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)